In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.decomposition import PCA

In [2]:
# data.csvのサイズが大きいので，分割して読み込む．
# あんまり読み込みすぎるとカーネルが落ちる．

def read_csv(data_file):
    # dataframe = pd.read_csv("data.csv")
    text_file_reader = pd.read_csv(data_file, chunksize=1000)
    return text_file_reader.get_chunk(200000)

In [3]:
# dataframe["ply"].hist(bins=12)

In [4]:
# DataFrame -> NumPy.ndarray

def split_data(dataframe):
    m_data = dataframe.values
    features = m_data[:,:-1]
    target = m_data[:,-1]
    return features, target

In [5]:
# Ridge回帰
def ridge_regression(features, target, alpha_val):
    ridge = Ridge(alpha=alpha_val)
    return ridge.fit(features, target)

In [6]:
# # 特徴量選択

# pca = PCA(n_components=0.999)
# features_pca = pca.fit_transform(features)
# print(features.shape[1], "->", features_pca.shape[1])


# # Ridge回帰（特徴量選択あり）

# ridge_pca = Ridge(alpha=0.1)
# model_ridge_pca = ridge_pca.fit(features_pca, target)
# model_ridge_pca.score(features_pca,target)

In [7]:
# 係数ベクトルの正規化・整数化
def normalize_coef(coef):
    pawn = np.mean(coef[5*64+8:6*64-8])
    return [int (c * 100 / pawn) for c in coef]

In [25]:
# 係数ベクトルの出力（標準出力）

def print_coef(coef):
    coef = [int(c) for c in coef]
    np.set_printoptions(precision=3,floatmode="fixed")
    indexes = ["King", "Queen", "Rook", "Bishop", "Knight", "Pawn"]
    for i in range(6):
        if (i == 5):
            mean = int(np.mean(coef[i*64+8:(i+1)*64-8]))
        else:
            mean = int(np.mean(coef[i*64:(i+1)*64]))
        print('{} (Mean: {})'.format(indexes[i], mean))
        for rank in range(7,-1,-1):
            for file in range(8):
                print('{:>4}'.format(coef[rank*8+i*64+file]),end=' ')
            print(' --- {:>4}'.format(int(np.mean(coef[rank*8+i*64:rank*8+i*64+8]))))
        print()
    color = 64 * 6
    print("Color")
    print(coef[color])
    castl = color + 1
    print("\nCastling")
    print(coef[castl:castl+2])
    print("\nPly")
    print(coef[-1])


In [26]:
# 係数ベクトルの出力（CSVファイル）
def print_coef_csv(coef_file, coef):
    np.savetxt(coef_file, coef, delimiter=',', fmt='%d')

In [27]:
game_mode = ['opening','midgame','endgame']

def learn_coef(game):
    data_file = "Data/data-" + game + ".csv"
    coef_file = "Data/coef-" + game + ".csv"
    print(game)
    dataframe = read_csv(data_file)
    features,target = split_data(dataframe)
    model = ridge_regression(features,target,0.01)
    coef = normalize_coef(model.coef_)
    print_coef(coef)
    print_coef_csv(coef_file, coef)

In [28]:
learn_coef(game_mode[0])

opening
King (Mean: 0)
   0    0    0    0    0    0    0    0  ---    0
   0    0    0    0    0    0    0    0  ---    0
   0    0    0    0    0    0    0    0  ---    0
   0    0    0    0  -37 -271  112   59  ---  -17
 -21  -77  259  -21 -123   98   49  -45  ---   14
 212   94  -26   -5  -41  -19  -12 -100  ---   12
 -12    3   17  -16  -37  -23   12  -23  ---   -9
  24   53   28  -67  -33  -16   16   -6  ---    0

Queen (Mean: 197)
 187  276  377   54  142  344  282  184  ---  230
 148  132  261  165  197  251  262  391  ---  225
 184  135  171  270  306  197  351  239  ---  231
 222  159  194  168  185  210  262  223  ---  202
 163  173  165  176  167  212  189  194  ---  179
 154  157  168  169  180  179  185  191  ---  172
 123  177  167  166  175  178  170  214  ---  171
 153  157  160  165  159  142  132  273  ---  167

Rook (Mean: 101)
 135  119  177   77   84  129  285  145  ---  143
 124  157  159  106  130  471  229  216  ---  199
 137  116  125  178  132  173  258  240 

In [29]:
learn_coef(game_mode[1])

midgame
King (Mean: 0)
 -86  -40  449  170   71   95    9    0  ---   83
 -87   10   46  116   74  245   18   -2  ---   52
 -21  -14   58   45  104   50    8   16  ---   30
  -3   40  -39   48   27  -31   15 -111  ---   -6
-116  -31  -14  -15  -20  -13  -42  -85  ---  -42
 -52  -36  -39  -23  -37  -39  -43  -67  ---  -42
 -25  -32  -27  -37  -29  -42  -30  -43  ---  -33
  -9  -22  -29  -68  -72  -55  -37  -41  ---  -41

Queen (Mean: 402)
 408  407  390  453  452  551  448  465  ---  446
 378  403  447  443  429  462  420  438  ---  427
 385  386  407  447  438  470  490  448  ---  433
 378  373  405  427  424  472  462  411  ---  419
 342  381  380  389  403  398  401  392  ---  385
 378  367  383  390  388  391  392  377  ---  383
 360  385  363  377  380  371  367  392  ---  374
 392  365  356  351  362  317  348  323  ---  351

Rook (Mean: 236)
 249  235  251  225  214  282  304  267  ---  253
 249  248  260  280  269  279  264  255  ---  263
 246  242  238  244  244  279  298  277 

In [30]:
learn_coef(game_mode[2])

endgame
King (Mean: 0)
  41  142  102   66   33  112   78   51  ---   78
 -23    5   80   78   32  141   81  -34  ---   45
  32   50    1   36   41   28   15   72  ---   34
 -55  -14   -5  -18   18  -13   12   -4  ---   -9
 -13  -43  -26  -33  -28  -17  -24  -28  ---  -26
 -35  -36  -35  -32  -38  -36  -24  -23  ---  -32
 -42  -28  -47  -41  -44  -26   -6  -33  ---  -33
 -84  -44  -49  -47  -59  -50  -54  -51  ---  -54

Queen (Mean: 582)
 558  526  465  484  604  586  709  556  ---  561
 522  549  612  553  657  620  582  672  ---  595
 495  560  595  679  627  607  572  578  ---  589
 492  587  563  640  603  632  636  603  ---  594
 622  586  583  585  603  599  570  572  ---  590
 610  612  550  607  559  569  619  613  ---  592
 439  551  588  616  584  496  607  463  ---  543
 619  577  620  609  534  595  669  523  ---  593

Rook (Mean: 331)
 341  366  336  336  328  369  334  339  ---  343
 341  342  342  348  336  371  342  357  ---  347
 332  346  352  333  343  336  316  322 